<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> COP-4</h1>
</center>

## <span style="color:blue">Nombre</span>: Francis Vargas Ferrer

## <span style="color:blue">Rol</span>: 201573026-1

# Librerías

In [1]:
import numpy as np
import scipy.linalg as spla

# Desarrollo

Para resolver el problema de mínimos cuadrados que se nos genera se utilizara el siguiente código, el cual es una modificación del que se encuentra en el material del curso[1]. La obtención de QR en este caso siempre es por el método modificado.

In [2]:
def QR(A):
    n, m = A.shape[::-1]
    rs = np.zeros((n,n))
    qs = np.zeros((m,n))
    for j in range(n):
        y = A[:, j]
        for i in range(0, j):
            rs[i, j] = qs[:,i]@y
            y = y-rs[i, j]*qs[:,i]
        rs[j, j] = np.linalg.norm(y)
        qs[:,j] = y/rs[j,j]
    return qs, rs

def lstsq_QR(A, b):
    Q, R = QR(A)
    x = spla.solve_triangular(R, np.dot(Q.T, b))
    return x

Modificaremos la el código visto en clases[2], para encontrar las casi-splines que pide el problema, añadiendo la condición de continuidad en la tercera derivada, la cual viene dada por:
$$
    S^{'''}_{i-1}(x_i) = S^{'''}_{i}(x_i) 
$$
Obteniendo la tercera derivada, se tiene que:

\begin{align}
    S(x_i) &= y_i+b_i(x−x_i)+c_i(x−x_i)^2+d_i(x−x_i)^3\\
    S^{'}(x_i) &= b_i + 2c_i(x-x_i)+3d_i(x-x_i)^2\\
    S^{''}(x_i) &= 2c_i + 6d_i(x-x_i)\\
    S^{'''}(x_i) &= 6d_i
\end{align}

In [3]:
def casi_spline(x, y, ws):
    #x: x-coordinates of points
    #y: y-coordinates of points
    n = len(x)
    A = np.zeros((4*n-7, 3*n-3))
    b = np.zeros(4*n-7)
    
    delta_x=np.diff(x)

    #Building the linear system of equations
    
    #1st property
    for i in np.arange(n-1):
        b[i]= y[i+1]-y[i]
        A[i,3*i:3*(i+1)] = [delta_x[i],delta_x[i]**2,delta_x[i]**3]
    #2nd property
    for i in np.arange(n-2):
        A[(n-1)+i,3*i:3*(i+1)+1]= [1, 2*delta_x[i], 3*delta_x[i]**2, -1]
    #3rd property
    for i in np.arange(n-2):
        A[(n-1)+(n-2)+i,3*i:3*(i+1)+2] = [0, 2, 6*delta_x[i], 0, -2]
    #Propiedad de la tercera derivada
    for i in np.arange(n-2):
        A[(n-1)+(n-2)+(n-2)+i,3*i:3*(i+1)+3] = [0, 0, 6, 0, 0, -6]
    
    #Obtenemos W
    W = np.zeros(4*n-7)
    W[:n-1] = [ws[0] for i in range(n-1)]
    W[n-1:2*n-3] = [ws[1] for i in range(n-2)]
    W[2*n-3:3*n-5] = [ws[2] for i in range(n-2)]
    W[3*n-5:4*n-7] = [ws[3] for i in range(n-2)]
    W = np.diag(W)
    
    #Solving the system
    WA = W@A
    Wb = W@b

    sol = lstsq_QR(WA, Wb)
    S = {'b':sol[::3],
         'c':sol[1::3],
         'd':sol[2::3],
         'x':x,
         'y':y,
         'A':A
        }
    return S

Función para evaluar el la casi spline, también encontrada en [2].

In [4]:
def cubic_spline_eval(xx,S,der=0):
    x=S['x']
    y=S['y']
    b=S['b']
    c=S['c']
    d=S['d']
    n=len(x)
    yy=np.zeros_like(xx)
    for i in np.arange(n-1):
        jj = np.where(np.logical_and(x[i]<=xx,xx<=x[i+1]))
        yy[jj]=y[i]+b[i]*(xx[jj]-x[i])+c[i]*(xx[jj]-x[i])**2+d[i]*(xx[jj]-x[i])**3
    return yy

Solo nos queda aplicar los métodos creados con anterioridad.

In [29]:
#Cargamos el dataset
data = np.load("Dataset1.npy")
x = data[:,0]
y = data[:,1]

#Definimos nuestro w
ws = np.array([1,0.1,1,1])

#Calculamos la casi spline
S = casi_spline(x,y, ws)

#### 1. Norma de Frobenius

In [30]:
A = S['A']
n, m = A.shape
frob = 0
for i in range(n):
    for j in range(m):
        frob += abs(A[i][j])**2
frob = np.sqrt(frob)
frob

28.92462461596844

#### Coeficiente $b_1$

In [31]:
b = S['b']
b[0]

1.3380029880045403

#### Error de interpolación

In [32]:
#Funcion original
f = lambda x: np.sin(x)
xs = np.linspace(0, 2*np.pi, 1000)
ys = f(xs)
#Evaluacion en el interpolador
ys_cs = cubic_spline_eval(xs, S)

np.linalg.norm(ys - ys_cs, np.inf)

0.05052887829316627

# Referencias

[1] https://github.com/sct-utfsm/INF-285/blob/master/material/06_minimos_cuadrados/qr.ipynb


[2] https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1/08_Interpolation_Splines.ipynb